# 크롤링
- 오늘 날짜부터 -n일차 까지 다음뉴스의 데이터를 긁어 오는 작업으로 시작합니다.

- 날짜를 거꾸로 수집을 합니다.

- 오늘로부터 며칠 전까지 이런식으로

- 다음 랭킹뉴스에서 50개까지 url을 긁어온 뒤

- 다시 for문을 이용해서 url을 접속해서 content를 긁어서 저장합니다.

- 이 함수의 역할은 여기까지입니다.

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime

In [2]:
# 하단 코드 구동 시 './datasets/craw/다음뉴스종합.csv' 파일 생성

In [3]:
# def daum(dates):

#     # 다음뉴스 헤드라인 긁어오기
#     files = pd.DataFrame()
#     for date in dates:
#         http=[]
#         print(date,'', 'Daum 접속 중')
#         httz = 'https://media.daum.net/ranking/popular/?regDate={}'.format(date)
#         res = requests.get(httz)
#         soup = BeautifulSoup(res.content, 'html.parser')
#         body = soup.select('#mArticle > div.rank_news > ul.list_news2')
#         body = body[0].find_all('a')


#         for i in range(len(body)):
#             t = body[i].get('href')
#             http.append(t)

#         # 중복제거
#         http = list(set(http))
        
#         for i in range(len(http)):
#             res = requests.get(http[i])
#             soup = BeautifulSoup(res.content, 'html.parser')
#             body = soup.select('.article_view')[0]

#             files = files.append(pd.DataFrame({
#                 'date':date,
#                 'title': soup.find('div', attrs={'class': 'head_view'}).h3.text,
#                 'content': " ".join(p.get_text() for p in body.find_all('p')),
#                 'link': http[i]
#             }, index=[i]))
#         time.sleep(15)

#         # 텍스트파일에 댓글 저장하기
#     files.to_csv('./datasets/craw/다음뉴스종합.csv',index=False,encoding='utf-8')
#     print('파일 저장 완료!')

# ----------------- 크롤링 실행 --------------------------
# # 현재로 부터 며칠 전까지 수집할 것인지 확인
# days = 1
# dates = [int(datetime.today().strftime('%Y%m%d')) - i for i in range(days)]
# daum(dates)

# 데이터 수집 끝
# 뉴스 content의 내용을 tfidf를 이용하여 유사도를 분석합니다.

<pre>
tfidf를 모르시면 한번 확인하시길 바랍니다.
이후 코드입니다.
원래는 뉴스데이터 원본을 바로 Tf-idf를 이용하고, 바로 코사인유사도를 이용하여 분석하였습니다.
하지만 조금 변경해서 명사추출로 먼저 데이터를 정제한 후에 모델에 입력합니다.
그 이후에 Tf-idf -> 코사인유사도로 문서간의 유사도를 분석합니다.

In [4]:
# self의 의미
# https://velog.io/@magnoliarfsit/RePython-1.-self-%EC%9D%B4%ED%95%B4%ED%95%98%EA%B8%B0

In [5]:
import numpy as np
dfsss = pd.DataFrame({"name": ['',' ',np.nan]})
dfsss['name'] = '-'+dfsss['name']+'-' 
dfsss.head()  

,name
0,--
1,- -
2,NaN


In [6]:
# ! pip install customized-konlpy   

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ckonlpy.tag import Twitter

#dataframe에 null값이 있는 경우 공백을 넣어 null값 제거
class TFIDF:
    def __init__(self,df, vo):
        self.okky_data_df = df
        self.word_dictionary = vo
        self.okky_data_df['noun_content'] = self.noun_extraction(self.okky_data_df['content'],
                                                         self.word_dictionary)

    # 전처리작업 (명사추출)
    def noun_extraction(self, multi_lines ,word_dictionary):
        tw = Twitter()
        tw.add_dictionary(word_dictionary, 'Noun')

        tokens_list = []

        for idx in range(len(multi_lines)):
            tokens_str = ''
            # multi_lines[0] : '안녕하세요 아무개씨' 
            # multi_lines[1] : '오늘 춥다' 
            # noun_list = ['안녕', '아무개']
            noun_list = tw.nouns(multi_lines[idx])
            for noun_str in noun_list:                    #['안녕', '아무개']
                tokens_str = tokens_str + ' ' + noun_str  #_안녕 아무개
  
            # 공백삭제
            tokens_str = tokens_str[1:]                   #안녕 아무개
            tokens_list.append(tokens_str)                # ["안녕 아무개", "오늘"]
        return tokens_list
    
    # Null 제거 함수
    def avoid_null(self,df, col):
        df[col] = df[col].fillna('')
        return df[col]

    # [전제집] 유사도 행렬을 반환해주는 함수
    def fit_tfidf(self):
        #결측처리
        self.okky_data_df['noun_content'] = self.avoid_null(self.okky_data_df, 'noun_content')
        #self.okky_data_df['noun_content'].fillna('', inplace=True)
        
        #tf-idf계산 후 출력
        tfidfVectorizer = TfidfVectorizer()
        self.tfidf_metrix = tfidfVectorizer.fit_transform(self.okky_data_df['noun_content'])
        return self.tfidf_metrix
    
    #입력되는 train의 질문과 질문 데이터셋의 코사인유사도 값 중 상위 50개 질문목록을 가져오는 함수
    def top10_indices(self, dm, idx):
        #입력된 데이터의 코사인유사도 계산
        cos_sim = linear_kernel(dm, dm)

        cos_sim_idx = list(enumerate(cos_sim[idx])) 
        cos_sim_idx = sorted(cos_sim_idx, key = lambda x : x[1], reverse = True)
        #상위 10개 항목을 가져옴
        topN_cos_sim_score = cos_sim_idx[1:11]
        tag_indices = [i_cos_sim[0] for i_cos_sim in topN_cos_sim_score]

        return tag_indices

word_dictionary는 명사추출에 앞서서 원하는 명사를 추출하기 위해서 사전 역할을 합니다.

위에서 긁어온 데이터를 다시 불러서 진행합니다.

In [8]:
searchstr = ['전세집']

new_df = pd.read_csv(r'./datasets/craw/다음뉴스종합.csv', encoding = "utf-8")

데이터를 입력하면 명사 추출까지는 자동으로 입력하고

그 이후 fit_tfidf 메서드를 불러서 유사도 metrix를 저장합니다.

In [9]:
TFIDF = TFIDF(new_df,  searchstr)

searchstr_cos_matrix = TFIDF.fit_tfidf() # [전제집] 유사도 행렬

c:\ai\pythonproject\venv\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [10]:
TFIDF.top10_indices(searchstr_cos_matrix, 0)

[9, 39, 28, 38, 24, 44, 43, 12, 11, 36]

마지막으로 코사인유사도로 순위가 나타난 문서들을 출력할 차례입니다.

In [12]:
# 50개까지만 출력합니다.
count = 3

for i in range(len(new_df[:count])):
    print(i, '/', len(new_df))
    #질문 제목과 데이터셋의 유사도를 10위까지 가져옴
    
    
    tit_10_q = new_df['content'].iloc[TFIDF.top10_indices(searchstr_cos_matrix, i)]
#     print(str(i),  okky_data.loc[i],"\n", tit_10_q)
    print(str(i),'<',new_df['title'][i],"> \n", tit_10_q)


0 / 50
0 < 돈도 축하도 필요없다..오늘 혼인 신고하고 왕실떠난 日공주 > 
 9     왕이 중국 외교담당 국무위원 겸 외교부장이 일본을 향해 “역사 인식과 대만 등은 중...
39     (서울=뉴스1) 김일창 기자 = 노태우 전 대통령이 26일 지병 악화로 서울 종로...
28     초기에 발견하기 어렵기 때문에 '침묵의 살인자'라 불리는 췌장암에 걸리는 젊은 여...
38     (서울=뉴스1) 김규빈 기자,음상준 기자,권영미 기자,이형진 기자,강승지 기자 =...
24      1970년대 중반의 일이다. 당시 공수여단장이었던 노태우 전 대통령은 난생 처음...
44     이인제 전 국회의원이 더불어민주당 대선 후보로 이재명 경기도지사가 선출된 것과 관...
43     [이데일리 박지혜 기자] 대한민국 제13대 대통령을 지낸 노태우 전 대통령이 26...
12     (시카고=연합뉴스) 김현 통신원 = 미국에서 많은 의문을 남긴 채 실종됐던 전도유...
11     [아시아경제 박준이 기자] 이재명 더불어민주당 대선 후보와 홍준표 국민의힘 의원 ...
36      # 내년 4월 결혼식을 위해 웨딩홀을 예약하려던 서울 서초동의 김모(32)씨. ...
Name: content, dtype: object
1 / 50
1 < '구속 갈림길' 영장심사 마치고 법원 나서는 손준성 검사 [TF사진관] > 
 37     (서울=뉴스1) 한유주 기자,최현만 기자 = 고발사주 의혹의 핵심인물이자 텔레그램...
27     김용판 의원은 이날 '돈다발' 사진과 관련해 "민주당은 돈다발 사진이 박철민 페이...
35     (서울=뉴스1) 이훈철 기자,김민성 기자 = 윤석열 국민의힘 대선 경선 후보가 2...
49     (서울=뉴스1) 허고운 기자,이밝음 기자 = 26일 국회 행정안전위원회 종합감사에...
48    기사내용 요약 공수처의 손준성 전 정책관 구속영장 청구 관련  변협 "방어권 보장 ...
33     윤석열 전 검찰총장의 이재명 더불어민주

정리

1.데이터를 크롤링한다.

2.명사만 따로 추출해서 저장한다.

3.Tf-idf를 이용해서 문서유사도를 구한다.(핵심 단어만 찾아낸다.)

4.구한 값을 이용해서 코사인유사도로 비슷한 문서를 찾는다.

